In [119]:
import pandas as pd
import numpy as np

import pandas_profiling

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer

In [120]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [121]:
train = train.drop(['Unnamed: 0', '일자'], axis=1)
test = test.drop(['Unnamed: 0', '일자'], axis=1)

In [122]:
def label_encode(train, test, columns):
    le = LabelEncoder()
    for column in columns:
        le.fit(pd.concat([train[column], test[column]]).drop_duplicates())
        train[column] = le.transform(train[column])
        test[column] = le.transform(test[column])

In [123]:
label_encode(train, test, ['요일'])

In [140]:
breakfast_train = [x for x in train['조식메뉴']]
lunch_train = [x for x in train['중식메뉴']]
dinner_train = [x for x in train['석식메뉴']]

breakfast_test = [x for x in test['조식메뉴']]
lunch_test = [x for x in test['중식메뉴']]
dinner_test = [x for x in test['석식메뉴']]

In [141]:
vectorizer = CountVectorizer()
breakfast_features = vectorizer.fit(breakfast_test + breakfast_train)
breakfast_train = vectorizer.transform(breakfast_train).toarray().flatten()
breakfast_test = vectorizer.transform(breakfast_test).toarray().flatten()

lunch_features = vectorizer.fit(lunch_test + lunch_train)
lunch_train = vectorizer.transform(lunch_train).toarray().flatten()
lunch_test = vectorizer.transform(lunch_test).toarray().flatten()

dinner_features = vectorizer.fit(dinner_test + dinner_train)
dinner_train = vectorizer.transform(dinner_train).toarray().flatten()
dinner_test = vectorizer.transform(dinner_test).toarray().flatten()


In [142]:
train['조식메뉴'] = pd.DataFrame(breakfast_train)
train['중식메뉴'] = pd.DataFrame(lunch_train)
train['석식메뉴'] = pd.DataFrame(dinner_train)

test['조식메뉴'] = pd.DataFrame(breakfast_test)
test['중식메뉴'] = pd.DataFrame(lunch_test)
test['석식메뉴'] = pd.DataFrame(dinner_test)


In [143]:
y = train['중식계']
x = train.drop('중식계', axis=1)

In [145]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(random_state=1)
model.fit(x, y)
res = model.predict(test)
submit = pd.read_csv('sample_submit.csv')
submit['중식계'] = res
submit.to_csv('submit.csv', index=False)